In [2]:
import json
import numpy as np
from pprint import pprint
import requests
import nltk.data
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

In [3]:
class JSONObject:
     def __init__(self, d):
         self.__dict__ = d

with open("articles_Accuracy.json", "r") as read_file:
    data = read_file.read()
    obj = json.loads(data,object_hook=JSONObject)      

In [4]:
from nltk.corpus import stopwords
stopworddic = set(stopwords.words('french'))
article=['a','au','aux','un','une','le','la','les','de','des','ce','cet','cette','ces','son','sa','ses','leur','leurs','mon','ma','mes','ton','ta','tes','notre','notres','votre','votres']
pronom=['je','tu','il','elle','nous','vous','ils','elles','on','y','en','se']
coordination=['mais','ou','et','donc','or','ni','car']
v=['aller','vais','vas','va','allez','allions','vont']
stopworddic.update(set(article),set(pronom),set(coordination),set(v))

In [5]:
lemmatizer = FrenchLefffLemmatizer()
def lemma(word):
    a=lemmatizer.lemmatize(word,'all')
    b=[x[1] for x in a]
    if 'nc' in b:
        return lemmatizer.lemmatize(word,'n')
    if 'v' in b:
        return lemmatizer.lemmatize(word,'v')
    return lemmatizer.lemmatize(word)

In [7]:
import re
rawtokens=nltk.word_tokenize(obj[0].body_fr)
metokens = [w.lower() for w in rawtokens if re.search('[a-zA-ZàâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]+',w)]
latokens=[]
for str in metokens:
    if(re.search(r'\w+\'',str)):
        str=re.sub(r'\w+\'', '', str) 
    latokens.append(str)
tokens = [w for w in latokens if w not in stopworddic]

In [10]:
tokenized_data=[]
for doc in obj :
    if len(doc.body_fr)>2 :
        text=doc.body_fr
        text=nltk.word_tokenize(text)
        text = [w.lower() for w in text if re.search('[a-zA-ZàâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]+',w)]
        latokens=[]
        for str in text:
            if(re.search(r'\w+\'',str)):
                str=re.sub(r'\w+\'', '', str) 
            latokens.append(str)
        tokens = [w for w in latokens if w not in stopworddic]
        tokenized_data.append(tokens)

In [11]:
print(tokenized_data[0])

['groupe', 'défense', 'veut', 'accompagner', 'jeunes', 'pousses', 'françaises', 'haut', 'potentiel', 'ici', 'banque', 'complète', 'offre', 'travaille', 'insérer', 'écosystème', 'développement', 'entreprises', 'société', 'générale', 'met', 'quête', 'nouvelles', 'pépites', 'banque', 'défense', 'sert', 'déjà', 'start-up', 'dont', 'jugées', 'particulièrement', 'prometteuses', 'annoncé', 'mardi', 'souhaitait', 'séduire', 'supplémentaires', 'horizon', 'ensemble', 'territoire', 'jargon', 'banque', 'pépite', 'entreprise', 'déjà', 'réussi', 'première', 'levée', 'fonds', 'moins', 'euros', 'accompagnée', 'incubateur', 'phare', 'fonds', 'capital-risque', 'créée', 'idéalement', 'entrepreneur', 'coup', 'essai', 'peut', 'agir', 'entreprises', 'déjà', 'suivies', 'banque', 'doit', 'encore', 'grandir', 'comme', 'recrutements', 'externes', 'parvenir', 'banque', 'peaufine', 'arsenal', 'signer', 'lundi', 'prochain', 'partenariat', 'bpifrance', 'vue', 'notamment', 'faire', 'dialoguer', 'conseillers', 'résea

In [12]:
from gensim import models,corpora
dictionary = corpora.Dictionary(tokenized_data)
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

D:\Users\Surface\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [14]:
lda_model = models.LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
lsi_model = models.LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [15]:
print("LDA Model:")
 
for idx in range(10):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(10):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

LDA Model:
Topic #0: 0.009*"the" + 0.007*"plus" + 0.005*"of" + 0.004*"and" + 0.004*"to" + 0.004*"comme" + 0.004*"luxembourg" + 0.003*"être" + 0.003*"france" + 0.003*"in"
Topic #1: 0.010*"plus" + 0.006*"start-up" + 0.004*"entreprises" + 0.004*"fonds" + 0.004*"comme" + 0.004*"innovation" + 0.004*"aussi" + 0.003*"millions" + 0.003*"faire" + 0.003*"france"
Topic #2: 0.006*"plus" + 0.005*"euros" + 0.004*"france" + 0.003*"millions" + 0.003*"groupe" + 0.003*"affaires" + 0.002*"aussi" + 0.002*"deux" + 0.002*"start-up" + 0.002*"paris"
Topic #3: 0.005*"plus" + 0.004*"start-up" + 0.004*"aussi" + 0.004*"entreprises" + 0.004*"innovation" + 0.004*"comme" + 0.004*"entreprise" + 0.003*"fait" + 0.003*"être" + 0.003*"tout"
Topic #4: 0.007*"plus" + 0.005*"entreprises" + 0.005*"entreprise" + 0.005*"aussi" + 0.004*"comme" + 0.004*"projet" + 0.004*"ans" + 0.004*"tout" + 0.004*"deux" + 0.003*"fait"
Topic #5: 0.006*"start-up" + 0.005*"numérique" + 0.005*"plus" + 0.004*"paris" + 0.004*"entreprises" + 0.004*"in